In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [2]:
import pyspark.sql.types as types
from pyspark.sql import functions as F

# df = spark.read.csv("./work/datasets/Activity Logs/", header=True, schema="timestamp TIMESTAMP, currentLocation STRING, participantId INT, currentMode STRING, hungerStatus STRING, sleepStatus STRING, apartmentId INT, availableBalance FLOAT, jobId INT, financialStatus STRING, dailyFoodBudget FLOAT, weeklyExtraBudget DOUBLE")
df = spark.read.csv("./work/datasets/Activity Logs/", header=True, schema=types.StructType([
    types.StructField('timestamp', types.TimestampType()),
    types.StructField('currentLocation', types.StringType()),
    types.StructField('participantId', types.IntegerType()),
    types.StructField('currentMode', types.StringType()),
    types.StructField('hungerStatus', types.StringType()),
    types.StructField('sleepStatus', types.StringType()),
    types.StructField('apartmentId', types.IntegerType()),
    types.StructField('availableBalance', types.FloatType()),
    types.StructField('jobId', types.IntegerType()),
    types.StructField('financialStatus', types.StringType()),
    types.StructField('dailyFoodBudget', types.FloatType()),
    types.StructField('weeklyExtraBudget', types.FloatType()),
])).\
    withColumn('currentLocationX', F.regexp_extract('currentLocation', 'POINT \(([^ ]+) ([^ ]+)\)', 1).cast('float')).\
    withColumn('currentLocationY', F.regexp_extract('currentLocation', 'POINT \(([^ ]+) ([^ ]+)\)', 2).cast('float')).\
    withColumn('date', F.to_date('timestamp'))

df.printSchema()
df.show(2)

root
 |-- timestamp: timestamp (nullable = true)
 |-- currentLocation: string (nullable = true)
 |-- participantId: integer (nullable = true)
 |-- currentMode: string (nullable = true)
 |-- hungerStatus: string (nullable = true)
 |-- sleepStatus: string (nullable = true)
 |-- apartmentId: integer (nullable = true)
 |-- availableBalance: float (nullable = true)
 |-- jobId: integer (nullable = true)
 |-- financialStatus: string (nullable = true)
 |-- dailyFoodBudget: float (nullable = true)
 |-- weeklyExtraBudget: float (nullable = true)
 |-- currentLocationX: float (nullable = true)
 |-- currentLocationY: float (nullable = true)
 |-- date: date (nullable = true)

+-------------------+--------------------+-------------+-----------+------------+-----------+-----------+----------------+-----+---------------+---------------+-----------------+----------------+----------------+----------+
|          timestamp|     currentLocation|participantId|currentMode|hungerStatus|sleepStatus|apartmentId|

In [3]:
df.write.partitionBy('date').mode('overwrite').parquet("./work/tobi/parquet/activity_logs2.parquet")